# Ackerberg sampling and onwards

In this notebook, we simulate out the games, create importance weights, and do a lot of other juicy stuff to get ready for estimation. There is a lot of looping and programming here, so this could get long, but let's try to keep things as transparent as possible. 

We should describe the essence of Ackerberg sampling here as well. 

The first steps are basically the same as usual:

In [1]:
import ipystata
import os
os.getcwd()

'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour'

In [ ]:
%%stata
cd C:\Users\mjbaker\Documents\GitHub\NewsHour
clear all
use AveragedDataDyno.dta
set more off

PID could not be obtained, can only be closed by using "close all".


The first thing we will do is generate (if it is not already there) a marker for who is in the game and who is not - **note**: this is perhaps where branchings of our work will begin in the future. So:

In [ ]:
%%stata
capture gen game= (lnews | otherl) & lnpps!=.
sort market stationid mt timeslot

Note that the above code reflects the idea that only those stations broadcasting local news or other local programming (entertainment programming) in the actual paper are "making decisions" at any given time. Also, we need to have a price variable for them as well. 

The usual next step is to pull everything we need into `Mata`, while also generating any additional variables that we might need. 

In [ ]:
%%stata
mata:
    st_view(lnews=.,.,"lnews")
    st_view(nnews=.,.,"nnews")
    st_view(otherl=.,.,"otherl")
    st_view(otherc=.,.,"otherc")
    st_view(si=.,.,"si")
    st_view(mt=.,.,"mt")
    st_view(id=.,.,"stationid")
    st_view(M=.,.,"market")
    st_view(time=.,.,"timeslot")
    st_view(game=.,.,"game")
    st_view(lnpps=.,.,"lnpps")
    st_view(pop=.,.,"ACS_HH") 
    st_view(l_ACS_HH=.,.,"l_ACS_HH")
    st_view(lnewsn=.,.,"lnewsn")
    st_view(otherln=.,.,"otherln")
    st_view(nnewsn=.,.,"nnewsn")
    st_view(othercn=.,.,"othercn")
    st_view(dln=.,.,"dln")

    lnviewn=ln(pop:*si)
    m=panelsetup(M,1)
end

Pull in the starting values and get them all set up. Note this requires a little bit of rearranging and some transformation to suit the way we entered things into the likelihood. We also set the number of draws at 20 - while this might seem low, it is hard to do more because the problem is so giant!

In [52]:
%%stata
mata:
    mata matuse DynoStarts23

    bo[,1::4]=bo[,1::4]:^2
    bo=bo[rows(bo),]


    betaDynoStart=bo[(5..23,27)]

    sdstav=exp(bo[24])
    sdmarv=exp(bo[25])
    sdmodv=exp(bo[26])
    
    draws=20
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     mata matuse DynoStarts23
(loading arate[1,27], bo[1,27], drawsbo[100,27], vals[100,1])

:     bo=bo[rows(bo),]

:     betaDynoStart=bo[(5..23,27)]

:     sdmarv=exp(bo[25])

:     sdmodv=exp(bo[26])

:     
:     draws=20

: end


In a previous workbook, we did a big reshape and some associated calculations to get the data in mata form, and in reshaped form. That is, so we have stations as observations, with each row representing a share, price, decision, etc. of dimension 6 - the number of time periods. 

This is commonly referred to as wide format in Stata, which, for some reason, I decided to mark by suffixing things with the `Long`. To keep things short, I gathered all this code (it is redundant) into a script called `MataReshape.do`. The resulting variables are defined exactly as they were in a previous workbook. 

In [53]:
%%stata
do MataFunctions\MataReshape.do


------------------------------------------------- mata (type end to exit) ------------------------------------------
:         
:         timeslots=6

:         lnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         nnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         otherlLong=J(rows(uniqrows(id)),timeslots,.)

:         othercLong=J(rows(uniqrows(id)),timeslots,.)

:         slnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         snnewsLong=J(rows(uniqrows(id)),timeslots,.)

:         sotherlLong=J(rows(uniqrows(id)),timeslots,.)

:         sothercLong=J(rows(uniqrows(id)),timeslots,.)

:         gameLong=J(rows(uniqrows(id)),timeslots,.)

:         popLong=J(rows(uniqrows(id)),timeslots,.)  /* Not necessary but for convenience and completeness */

:         l_ACS_HHLong=J(rows(uniqrows(id)),timeslots,.)

:         
:         siLong=J(rows(uniqrows(id)),timeslots,.)

:         lnewsLongLag=J(rows(uniqrows(id)),timeslots,.)

:         nnewsLongLag=J(rows(uniqrows(id)),timeslots,.)

## Viewership error terms

That having been done, we now have to recover actual viewership errors, given our model parameters. More correctly, what we really need is the utility index for viewership. This is pretty easy to calculate given our model estimates. If we have (ignoring subscripts, etc. just to give the basic idea):

$$
\ln(s_i)-\ln(s_0) = \mu \ln(s_{ig})+U_i
$$

where $s_{ig}$ is the share of $i$ in $i$'s programming group. 

We can get $U_i=X\beta_v+\epsilon$ by doing:

$$
U_i = \ln(s_i)-\ln(s_0)-\mu \ln (s_{ig})
$$

In St(M)ata, we calculate the within group share (even though we have it - all these redundancies are good for checking things are working), and then calculate this linear viewership term. We also include a calculation that includes everything in the $X\beta$ vector, including error terms. 

In [54]:
%%stata
mata:
    swgLong=siLong:/(slnewsLong:*lnewsLong:+
        snnewsLong:*nnewsLong:+
        sotherlLong:*otherlLong:+
        sothercLong:*othercLong)
    
    lnswgLong=ln(swgLong)   

    dln=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong))    
    
    XBVngLong=dln-bo[1]*lnswgLong:*lnewsLong-bo[2]*lnswgLong:*otherlLong-
        bo[3]*lnswgLong:*nnewsLong-bo[4]*lnswgLong:*othercLong
                
    xBpUE=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong)):-
        bo[1]:*lnswgLong:*lnewsLong:-
        bo[2]:*lnswgLong:*otherlLong-
        bo[3]:*lnswgLong:*nnewsLong:-
        bo[4]:*lnswgLong:*othercLong
                
    xBDyno=J(rows(lnewsLong),cols(lnewsLong),0)
    for (i=1;i<=cols(lnewsLong);i++) {
        xDyno=lnewsLong[,i],otherlLong[,i],nnewsLong[,i],
            lnewslnews[,i],lnewsnnews[,i],nnewslnews[,i],nnewsnnews[,i],
            siLongLag[,i],siXlnln[,i],siXlnnn[,i],siXnnln[,i],siXnnnn[,i],
            lnewsRT[,i],nnewsRT[,i],l_ACS_HHLong[,i],lnewsnLong[,i],otherlnLong[,i],
            nnewsnLong[,i],othercnLong[,i],J(rows(lnewsLong),1,1)
        xBDyno[,i]=xDyno*betaDynoStart'
    }              
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     swgLong=siLong:/(slnewsLong:*lnewsLong:+
>         snnewsLong:*nnewsLong:+
>         sotherlLong:*otherlLong:+
>         sothercLong:*othercLong)

:     
:     lnswgLong=ln(swgLong)   

:     
:     XBVngLong=dln-bo[1]*lnswgLong:*lnewsLong-bo[2]*lnswgLong:*otherlLong-
>         bo[3]*lnswgLong:*nnewsLong-bo[4]*lnswgLong:*othercLong

:                 
:     xBpUE=ln(siLong):-ln(1:-(slnewsLong:+snnewsLong:+sotherlLong:+sothercLong)):-
>         bo[1]:*lnswgLong:*lnewsLong:-
>         bo[2]:*lnswgLong:*otherlLong-
>         bo[3]:*lnswgLong:*nnewsLong:-
>         bo[4]:*lnswgLong:*othercLong

:                 
:     xBDyno=J(rows(lnewsLong),cols(lnewsLong),0)

:     for (i=1;i<=cols(lnewsLong);i++) {
>         xDyno=lnewsLong[,i],otherlLong[,i],nnewsLong[,i],
>             lnewslnews[,i],lnewsnnews[,i],nnewslnews[,i],nnewsnnews[,i],
>             siLongLag[,i],siXl

In [55]:
%%stata
mata: XBVngLong[1::10,]
mata: hasmissing(XBVngLong)


                   1              2              3              4              5              6
     +-------------------------------------------------------------------------------------------+
   1 |  -8.986125165   -8.393051313   -8.731954925   -8.434416937   -6.958572773   -6.845780367  |
   2 |  -8.151462126   -8.357771758   -8.689353412   -8.615667796   -7.966964742   -7.966263611  |
   3 |  -8.235964752   -8.282701206   -8.344938115   -8.461468037   -8.280771568   -8.178466753  |
   4 |  -9.450680982   -9.175425169   -9.431472338   -8.853539716   -9.088035641   -9.139304896  |
   5 |  -9.178516731   -9.091969584   -8.417490835   -8.300316541   -8.366618495    -8.26941982  |
   6 |  -8.637199531    -8.51059762   -9.197810613    -8.99451705   -7.609286268   -7.513386088  |
   7 |  -8.743392727   -8.628013358   -8.730313932   -8.358525579   -8.014242822   -7.948298291  |
   8 |  -8.185973301    -7.86377515   -7.539445107   -7.446105978   -7.031060774   -6.766924731  |
   9 |    -7

These seem about right. so, now what we can do is start drawing random effects and all that. To do this, we can write a little function called `paneldraw()` which draws common values across a user-specified panel identifier. I've written this function so that it works with the id only, and doesn't require the data to be sorted. 

Since we will need this function again, we will do our adopted trick of writing the results to a file and then running the file. 

In [56]:
CWD = os.getcwd()
os.chdir(CWD + '\\MataFunctions')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\mjbaker\\Documents\\GitHub\\NewsHour\\MataFunctions\\MataFunctions'

In [57]:
%%writefile paneldraw.do
mata: 
    real matrix paneldraw(real matrix id, real scalar draws)
    {
        rowsToDraw=uniqrows(id)

        U=J(rows(id),draws,.)
        for (i=1;i<=rows(rowsToDraw);i++) {
            obs=mm_which(rowsToDraw[i]:==id)
            D=runiform(1,draws)
            U[obs,]=J(rows(obs),1,D)
        }
        return(U)
    }
end
end

Overwriting paneldraw.do


In [58]:
%%stata
do MataFunctions\paneldraw.do


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     real matrix paneldraw(real matrix id, real scalar draws)
>     {
>         rowsToDraw=uniqrows(id)
> 
>         U=J(rows(id),draws,.)
>         for (i=1;i<=rows(rowsToDraw);i++) {
>             obs=mm_which(rowsToDraw[i]:==id)
>             D=runiform(1,draws)
>             U[obs,]=J(rows(obs),1,D)
>         }
>         return(U)
>     }

: end
--------------------------------------------------------------------------------------------------------------------

end of do-file


**Note that we are now using stata in one directory, but Python is working in the directory `MataFunctions` **

Note that this just provides uniform random variates with common values across panel variables. We just use our estimated parameters and the usual inverse-normal transformation to get the actual values. We then reshape these into a useable form and call the results `UvsLong` and `UvmtLong` - the former being the station random effect, the latter being the market random effect. 

As the last part of the code block, we draw a purely idiosynchratic error term. 

Note that the stata package `moremata` is required for this step.

In [59]:
%%stata
mata:
    iev1=paneldraw(id,draws)
    iev2=paneldraw(mt,draws)
    iev1=sdstav*invnormal(iev1)
    iev2=sdmarv*invnormal(iev2)

    UvsLong=J(rows(lnewsLong),0,0)
    UvmtLong=J(rows(lnewsLong),0,0)
    
    for (i=1;i<=draws;i++) {
        UvsLong=UvsLong,colshape(iev1[,i],timeslots)
        UvmtLong=UvmtLong,colshape(iev2[,i],timeslots)
    }
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     iev1=paneldraw(id,draws)

:     iev2=paneldraw(mt,draws)

:     iev1=sdstav*invnormal(iev1)

:     iev2=sdmarv*invnormal(iev2)

:     UvmtLong=J(rows(lnewsLong),0,0)

:     
:     for (i=1;i<=draws;i++) {
>         UvsLong=UvsLong,colshape(iev1[,i],timeslots)
>         UvmtLong=UvmtLong,colshape(iev2[,i],timeslots)
>     }

: end


To keep stuff outside the loop outside the cell:

In [62]:
%%stata
mata: 
    UvmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodv
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     UvmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodv

: end


The next thing to be done is to calculate what amount to "observed" error terms for the viewership model (as opposed to simulated error terms from things we haven't observed). These are basically whatever is left over after fitting the model.
I will calculate all this out, and then expand things into a sample that conforms with the draws. 

However, note that the end result **DOES** vary, because we do not observe the division of the error term into market-time, station, and idiosyncratic effects. The last line subtracts out our sampled two level random effects.

In [63]:
%%stata
mata:
    UvmodObsLong=xBpUE:-xBDyno
    UvmodObsLong=J(1,draws,1)#UvmodObsLong:-UvsLong:-UvmtLong
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     UvmodObsLong=xBpUE:-xBDyno

:     UvmodObsLong=J(1,draws,1)#UvmodObsLong:-UvsLong:-UvmtLong

: end


## Price error terms

So, we now have a full set of error terms for the viewership model. We will now turn to dealing with the price model. The first step is loading up our estimated price model and its estimated coefficients, and then transforming and collecting the estimated variance terms. Here we go:

In [64]:
%%stata
mata:
    mata matuse betaPDynoStarts, replace
    
    sdstap=exp(bpo[7])
    sdmarp=exp(bpo[8])
    sdmodp=exp(bpo[9])    
    
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     mata matuse betaPDynoStarts, replace
(loading bpo[1,10], drawsbpo[200,10])

:     
:     sdstap=exp(bpo[7])

:     sdmarp=exp(bpo[8])

:     sdmodp=exp(bpo[9])    

:     
: end


Now, we need to get random effects in order:

In [65]:
%%stata
mata:
    iep1=paneldraw(id,draws)
    iep2=paneldraw(mt,draws)
    iep1=sdstap*invnormal(iep1)
    iep2=sdmarp*invnormal(iep2)	

    UpsLong=J(rows(lnewsLong),0,0)
    UpmtLong=J(rows(lnewsLong),0,0)
    
    for (i=1;i<=draws;i++) {
        UpsLong=UpsLong,colshape(iep1[,i],timeslots)
        UpmtLong=UpmtLong,colshape(iep2[,i],timeslots)
    }
    
    UpmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodp
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     iep1=paneldraw(id,draws)

:     iep2=paneldraw(mt,draws)

:     iep1=sdstap*invnormal(iep1)

:     iep2=sdmarp*invnormal(iep2)

:     UpmtLong=J(rows(lnewsLong),0,0)

:     
:     for (i=1;i<=draws;i++) {
>         UpsLong=UpsLong,colshape(iep1[,i],timeslots)
>         UpmtLong=UpmtLong,colshape(iep2[,i],timeslots)
>     }

:     
:     UpmodLong=invnormal(runiform(rows(lnewsLong),timeslots*draws))*sdmodp

: end


## **Point of deviation!**

At this point, we are deviating from the model as it currently appears in the paper. Given the comments of discussants at various presentations, we felt that it would be useful to take heed to one critique, given that we do not have data on the relative costs of broadcasting local news or entertainment, and that people feel that our results are "too large" in terms of welfare effects. 

One reason this might be the case is that we have been assuming throughout that **_any_** local station could switch to local news. This might be excessive, so we modifying things to allow only stations that have broadcasted local news at some point during the day to broadcast local news (i.e., these are the stations that have a demonstrated ability to broadcast news). 

Here is our original code that does this:

In [66]:
%%stata
mata:
    gameMarker = rowsum(gameLong):>0
    sum(gameMarker)
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     gameMarker = rowsum(gameLong):>0

:     sum(gameMarker)
  623

: end


We see that 623 stations are decision-makers in this case. We adjust to zero firms that never broadcast local news (they are now "out of the game". We also create a conformable version of `decCounter`, which counts the number of periods in which a station can make decisions. We also adjust our `gameLong` marker. 

How many of these stations never broadcast local news? 

In [67]:
%%stata
mata:
    lnewsSome  = rowsum(lnewsLong):>0
    gameMarker = gameMarker:*lnewsSome
    gameLong   = gameLong:*gameMarker
    decCounter = rowsum(gameLong)
    
    sum(gameMarker)
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     lnewsSome  = rowsum(lnewsLong):>0

:     gameMarker = gameMarker:*lnewsSome

:     gameLong   = gameLong:*gameMarker

:     decCounter = rowsum(gameLong)

:     
:     sum(gameMarker)
  400

: end


So, we see that the set of decision-making stations per our model is reduced by about 1/3. Let's proceed along these lines now. 

The basic idea now is to create for each station a set of arrays that store error terms and draws, potential strategy sets (remembering that national news is fixed), and simulated shares from unilateral deviations. We will carry these around in three associative arrays (which can be saved as-is for use in the sequel). 

In [68]:
%%stata
mata:
    Bcs      =asarray_create("real", 2)
    sharesBcs=asarray_create("real", 3)
    uVinfo   =asarray_create("real", 3) 
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     Bcs      =asarray_create("real", 2)

:     sharesBcs=asarray_create("real", 3)

:     uVinfo   =asarray_create("real", 3) 

: end


We also need a little function that can delineate all the potential strategies as they consist of ones and zeros. The function `stratmat()` does this. To give credit where credit is due, I believe I borrowed the code from an old Stata Journal piece by Nick Cox. 

I also give an example with four possible strategies just to be sure it's working. 

In [69]:
%%writefile stratmat.do
mata:
real matrix stratmat(real scalar x)
{
    real matrix M,MM,V
    real colvector p
    real scalar info,j

    V=J(x,x,1)
    V=J(rows(V),1,0),lowertriangle(V)
    MM=J(rows(V),1,.)
    for (j=1;j<=cols(V);j++) {
        info=cvpermutesetup(V[.,j])
        M=J(rows(V),1,.)
        while ((p=cvpermute(info)) !=J(0,1,.)) {
            M=M,p
        }
    M=M[.,2::cols(M)]
    MM=MM,M
    }
    MM=MM[.,2::cols(MM)]
    
    return(MM)
} 
end
end

Overwriting stratmat.do


In [70]:
%%stata
do MataFunctions\stratmat.do
mata: stratmat(4)


------------------------------------------------- mata (type end to exit) ------------------------------------------
: real matrix stratmat(real scalar x)
> {
>     real matrix M,MM,V
>     real colvector p
>     real scalar info,j
> 
>     V=J(x,x,1)
>     V=J(rows(V),1,0),lowertriangle(V)
>     MM=J(rows(V),1,.)
>     for (j=1;j<=cols(V);j++) {
>         info=cvpermutesetup(V[.,j])
>         M=J(rows(V),1,.)
>         while ((p=cvpermute(info)) !=J(0,1,.)) {
>             M=M,p
>         }
>     M=M[.,2::cols(M)]
>     MM=MM,M
>     }
>     MM=MM[.,2::cols(MM)]
>     
>     return(MM)
> } 

: end
--------------------------------------------------------------------------------------------------------------------

end of do-file
        1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16
    +---------------------------------------------------------------------------------+
  1 |   0    1    0    1    1    1    0    0    0    1    1    1    0    0    0    1  |

Having created the associative arrays, we now forge onwards and pick out our competitors' action vectors ids, and total number of periods in which decisions can be made. Note that if only five decisions are made, we know that a national news broadcast occurs at some point in the broadcast evening. 

In [71]:
%%stata
mata:
    gameIds=select(statIdLong,gameMarker)
    lnewsBcs=select(lnewsLong,gameMarker)
    nnewsBcs=select(nnewsLong,gameMarker)
    otherlBcs=select(otherlLong,gameMarker)
    totalDecs=select(decCounter,gameMarker)
    
    for (i=1;i<=rows(gameIds);i++) {
        if (totalDecs[i]==6) {
            strats=stratmat(6)'
            asarray(Bcs,(gameIds[i,1],1),strats)
            asarray(Bcs,(gameIds[i],2),1:-strats)
            asarray(Bcs,(gameIds[i],3),J(rows(strats),cols(strats),0))
        }
        if (totalDecs[i]==5) {
            nnewsPos=mm_which(nnewsBcs[i,1::6])
            stratsFree=stratmat(5)'
            if (nnewsPos==1) lnewsp=J(rows(stratsFree),1,0),stratsFree
            else if (nnewsPos==6) lnewsp=stratsFree,J(rows(stratsFree),1,0)
            else lnewsp=stratsFree[,1::nnewsPos-1],J(rows(stratsFree),1,0),stratsFree[,nnewsPos::5]
            otherlp=1:-lnewsp
            otherlp[,nnewsPos]=J(rows(otherlp),1,0)
            nnewsp=J(rows(lnewsp),6,0)
            nnewsp[,nnewsPos]=J(rows(lnewsp),1,1)
            asarray(Bcs,(gameIds[i],1),lnewsp)
            asarray(Bcs,(gameIds[i],2),otherlp)
            asarray(Bcs,(gameIds[i],3),nnewsp)
        }
    }
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     gameIds=select(statIdLong,gameMarker)

:     lnewsBcs=select(lnewsLong,gameMarker)

:     nnewsBcs=select(nnewsLong,gameMarker)

:     otherlBcs=select(otherlLong,gameMarker)

:     totalDecs=select(decCounter,gameMarker)

:     
:     for (i=1;i<=rows(gameIds);i++) {
>         if (totalDecs[i]==6) {
>             strats=stratmat(6)'
>             asarray(Bcs,(gameIds[i,1],1),strats)
>             asarray(Bcs,(gameIds[i],2),1:-strats)
>             asarray(Bcs,(gameIds[i],3),J(rows(strats),cols(strats),0))
>         }
>         if (totalDecs[i]==5) {
>             nnewsPos=mm_which(nnewsBcs[i,1::6])
>             stratsFree=stratmat(5)'
>             if (nnewsPos==1) lnewsp=J(rows(stratsFree),1,0),stratsFree
>             else if (nnewsPos==6) lnewsp=stratsFree,J(rows(stratsFree),1,0)
>             else lnewsp=stratsFree[,1::nnewsPos-1],J(rows(stratsFree),1,0),str

First, let's set up the variables that we need to do this. We have the following variables that need to be set up before the loop begins, and we also need to load up our functions for calculating shares, which we called `sharemakers.do`. Since we wrote that to a file, we can really just do this as follows...

In [72]:
%%stata
mata:
    mLong   = panelsetup(marketIdLong,1)
    counter = 1
    sniffTest = J(0,3,.)
end

do MataFunctions\sharemakers.do


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     mLong   = panelsetup(marketIdLong,1)

:     counter = 1

:     sniffTest = J(0,3,.)

: end
--------------------------------------------------------------------------------------------------------------------
------------------------------------------------- mata (type end to exit) ------------------------------------------
:     real matrix eshares_up(Uv,lnews,otherl,nnews,otherc,sigma_l,sigma_o,sigma_n,sigma_c)
>     {
>         sigg=lnews:*sigma_l:+otherl:*sigma_o:+nnews:*sigma_n:+otherc:*sigma_c
>         lu=exp(Uv:/(1:-sigg))
>         lu_l=colsum(lnews:*lu)
>         lu_n=colsum(nnews:*lu)
>         lu_o=colsum(otherl:*lu)
>         lu_c=colsum(otherc:*lu)
>         swg=lu:/(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc)
>         share=swg:*(lu_l:*lnews:+lu_n:*nnews:+lu_o:*otherl:+lu_c:*otherc):^(1:-sigg):/
>                   (1:+lu_l:^(1:-sigma_l):+

## Unilateral shares from deviating, given draws 

Now, the rubber is really going to hit the road. In the next block of code, we have a series of nested loops that does this. Let's make a schematic of levels for this so we can see what is going on. 

* loop over draws
   * loop over markets
       - calculate the number of players and only "do something" if there are actually players in the market
       - if we have players:
           * get their data and their positions (broadcast decisions, error terms, X variables...
           * loop over each player, and:
              - calculate out XB and hypothetical XB's for each possible broadcast sequence
              - get XB's for everyone else in the market
              - simulate shares for each potential unilateral deviation. 
              - plop strategies and coincident simulated unilater deviations into the array
              
              
   

In [73]:
%%stata
mata: 
    for (c=1;c<=draws;c++) {

        for (i=1;i<=rows(mLong);i++) {
            gameMarkerp=panelsubmatrix(gameMarker,i,mLong)
            playersp=colsum(gameMarkerp)
            if (playersp>0) {
                statIdLongp=panelsubmatrix(statIdLong,i,mLong)
                lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
                nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
                otherlLongp=panelsubmatrix(otherlLong,i,mLong)
                othercLongp=panelsubmatrix(othercLong,i,mLong)
                nsToChange=select(statIdLongp,gameMarkerp)
                UvmtLongp=panelsubmatrix(UvmtLong,i,mLong)
                UvmodLongp=panelsubmatrix(UvmodLong,i,mLong)
                UvsLongp=panelsubmatrix(UvsLong,i,mLong)
                XBVngLongp=panelsubmatrix(XBVngLong,i,mLong)
                UvmodObsLongp=panelsubmatrix(UvmodObsLong,i,mLong)
                l_ACS_HHLongp=panelsubmatrix(l_ACS_HHLong,i,mLong)
                
                for (k=1;k<=rows(nsToChange);k++) {
                    pId=nsToChange[k]
                    place=mm_which(statIdLongp:==pId)
                    lnewsHat=asarray(Bcs,(pId,1))
                    otherlHat=asarray(Bcs,(pId,2))
                    nnewsHat=asarray(Bcs,(pId,3))
                    sharesToPlace=J(rows(lnewsHat),6,0)
                    soPlace=J(rows(lnewsHat),6,0)
                    sgPlace=J(rows(lnewsHat),6,0)
                    lnewsOrig=lnewsLongp[place,]
                    otherlOrig=otherlLongp[place,]
                    XBVplaceHold=J(rows(lnewsHat),cols(lnewsHat),.)
                    XBVplaceHoldMean=J(rows(lnewsHat),cols(lnewsHat),.)
                    
                    for (q=1;q<=rows(lnewsHat);q++) {
                        lnewsLongp[place,]=lnewsHat[q,]
                        otherlLongp[place,]=otherlHat[q,]
                        nnewsLongp[place,]=nnewsHat[q,]
                        lnewsLongLagp=J(rows(lnewsLongp),1,0)
                        nnewsLongLagp=J(rows(lnewsLongp),1,0)
                        otherlLongLagp=J(rows(lnewsLongp),1,0)
                        siLagp=J(rows(lnewsLongp),1,0)
                        totlnewsp=J(rows(lnewsLongp),1,0)
                        totnnewsp=J(rows(nnewsLongp),1,0)
    
                        lnewsnLongp=lnewsLongp:*ln(1:+colsum(lnewsLongp))
                        otherlnLongp=otherlLongp:*ln(1:+colsum(otherlLongp))
                        nnewsnLongp=nnewsLongp:*ln(1:+colsum(nnewsLongp))
                        othercnLongp=othercLongp:*ln(1:+colsum(othercLongp))

                        for (t=1;t<=timeslots;t++) {
                            if (t!=1) siLagp=ln(siLagp)
                            XV=lnewsLongp[,t],otherlLongp[,t],nnewsLongp[,t],
                                lnewsLongLagp:*lnewsLongp[,t],
                                nnewsLongLagp:*lnewsLongp[,t],
                                lnewsLongLagp:*nnewsLongp[,t],
                                nnewsLongLagp:*nnewsLongp[,t],
                                siLagp,
                                siLagp:*lnewsLongLagp:*lnewsLongp[,t],
                                siLagp:*nnewsLongLagp:*lnewsLongp[,t],
                                siLagp:*lnewsLongLagp:*nnewsLongp[,t],
                                siLagp:*nnewsLongLagp:*nnewsLongp[,t],
                                lnewsLongp[,t]:*ln(1:+totlnewsp),nnewsLongp[,t]:*ln(1:+totnnewsp),
                                l_ACS_HHLongp[,t],lnewsnLongp[,t],otherlnLongp[,t],
                                nnewsnLongp[,t],othercnLongp[,t],J(rows(lnewsLongp),1,1)

                                XBV=XV*betaDynoStart':+UvmtLongp[,counter+t-1]:+
                                    UvsLongp[,counter+t-1]:+UvmodObsLongp[,counter+t-1]
                                if (lnewsHat[q,t]!=lnewsOrig[t]) {
                                    XBV[place,]=XV[place,]*betaDynoStart':+UvmtLongp[place,counter+t-1]:+
                                        UvsLongp[place,counter+t-1]:+UvmodLongp[place,counter+t-1]
                                }

                            XBVplaceHold[q,t]=XBV[place]
                            XBVplaceHoldMean[q,t]=XBV[place]-UvmodLongp[place,counter+t-1]

                            sharesP=esharesStable(XBV,lnewsLongp[,t],otherlLongp[,t],nnewsLongp[,t],
                                othercLongp[,t],bo[1],bo[2],bo[3],bo[4])
                            sharesToPlace[q,t]=sharesP[place,]
                            soPlace[q,t]=1-sum(sharesP)

                            slPlace=sum(lnewsLongp[,t]:*sharesP)
                            snPlace=sum(nnewsLongp[,t]:*sharesP)
                            solPlace=sum(otherlLongp[,t]:*sharesP)
                            scPlace=sum(othercLongp[,t]:*sharesP)

                            sgPlace[q,t]=lnewsLongp[place,t]:*slPlace:+
                                nnewsLongp[place,t]:*snPlace:+
                                otherlLongp[place,t]:*solPlace:+
                                othercLongp[place,t]:*scPlace

                            siLagp=sharesP
                            totlnewsp=totlnewsp:+J(rows(lnewsLongp),1,colsum(lnewsLongp[,t]:*sharesP))
                            totnnewsp=totnnewsp:+J(rows(nnewsLongp),1,colsum(nnewsLongp[,t]:*sharesP))
                            lnewsLongLagp=lnewsLongp[,t]
                            nnewsLongLagp=nnewsLongp[,t]
                            otherlLongLagp=otherlLongp[,t]
                        }
                        
                        asarray(sharesBcs,(pId,c,1),sharesToPlace)
                        asarray(sharesBcs,(pId,c,2),soPlace)
                        asarray(sharesBcs,(pId,c,3),sgPlace)
                        asarray(Bcs,(pId,4),XBVplaceHold)
                        asarray(Bcs,(pId,5),XBVplaceHoldMean)
                        lnewsLongp[place,]=lnewsOrig
                        otherlLongp[place,]=otherlOrig
                        sniffTest=sniffTest \ (c,mean(rowsum(sharesToPlace)),rowsum(siLong[k,]))

                    }
                }
            }
        }
        counter=counter+timeslots
    }
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     for (c=1;c<=draws;c++) {
> 
>         for (i=1;i<=rows(mLong);i++) {
>             gameMarkerp=panelsubmatrix(gameMarker,i,mLong)
>             playersp=colsum(gameMarkerp)
>             if (playersp>0) {
>                 statIdLongp=panelsubmatrix(statIdLong,i,mLong)
>                 lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
>                 nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
>                 otherlLongp=panelsubmatrix(otherlLong,i,mLong)
>                 othercLongp=panelsubmatrix(othercLong,i,mLong)
>                 nsToChange=select(statIdLongp,gameMarkerp)
>                 UvmtLongp=panelsubmatrix(UvmtLong,i,mLong)
>                 UvmodLongp=panelsubmatrix(UvmodLong,i,mLong)
>                 UvsLongp=panelsubmatrix(UvsLong,i,mLong)
>                 XBVngLongp=panelsubmatrix(XBVngLong,i,mLong)
>                 UvmodObsLongp=panelsubmat

At this point, waiting around for programs to run can be a little frustrating. So, we made a script that at least fills us in that things are moving along. It will log whenever an iteration is completed, and send me a text message when done. 

The idea is that we have a ``listener.py`` program running in the background. This program detects changes in a folder - one of which can be writing a file from stata. So, we will have Stata write and delete a file at some prespecified time in the program. 

Given the `listener` is listening for changes, the following little block of code sends a text message, notes changes, etc. 


Now, we have to assembled all the sampling weights, distributions, etc. of all the data. We will pile all of this into a whole bunch of different places. So:

In [74]:
%%stata
mata:
    uv=J(rows(statIdLong),0,.)
    uvg     =J(rows(statIdLong),0,.)
    uvsi    = J(rows(statIdLong),0,.)
    uvso    = J(rows(statIdLong),0,.)
    uvsg    = J(rows(statIdLong),0,.)
    uvre1   = J(rows(statIdLong),0,.)
    uvre2   = J(rows(statIdLong),0,.)
    uvre1g  = J(rows(statIdLong),0,.)
    uvre2g  = J(rows(statIdLong),0,.)
   
    looper  = uniqrows(asarray_keys(sharesBcs)[,1])
    counter = 1
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     uv=J(rows(statIdLong),0,.)

:     uvg     =J(rows(statIdLong),0,.)

:     uvsi    = J(rows(statIdLong),0,.)

:     uvso    = J(rows(statIdLong),0,.)

:     uvsg    = J(rows(statIdLong),0,.)

:     uvre1   = J(rows(statIdLong),0,.)

:     uvre2   = J(rows(statIdLong),0,.)

:     uvre1g  = J(rows(statIdLong),0,.)

:     uvre2g  = J(rows(statIdLong),0,.)

:    
:     looper  = uniqrows(asarray_keys(sharesBcs)[,1])

:     counter = 1

: end


## Ackerberg-Keane-Wolpin importance sampling weights

### Part one - linear component

Conceptually, there are really two aspects to computing the importance samplin weights. One aspect just involves computing the weights of all the drawn linear terms that we have. This is fairly easy.

Of course, the second part is much, much harder. This requires that we create some sort of sampling weight based on the existence of multiple equilibria. We will get to that in a moment. 

One other subroutine we need is a means of calculating in stable fashion the log-density value of a normal random variable. We use the `lnnd()` function for that:

 

In [75]:
%%writefile lnnd.do
mata
    real matrix lnnd(y,m,lnsd) return(-(y:-m):^2:/(2:*exp(lnsd):^2):-ln(2*pi())/2:-lnsd)   
end
end

Overwriting lnnd.do


In [76]:
%%stata
do MataFunctions\lnnd.do


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     real matrix lnnd(y,m,lnsd) return(-(y:-m):^2:/(2:*exp(lnsd):^2):-ln(2*pi())/2:-lnsd)   

: end
--------------------------------------------------------------------------------------------------------------------

end of do-file


In [78]:
%%stata
mata:
    for (c=1;c<=draws;c++) {
        ackToAdd=J(rows(statIdLong),timeslots,.)
        ackToAddg=J(rows(statIdLong),timeslots,.)
        ackToAddsi=J(rows(statIdLong),timeslots,.)
        ackToAddso=J(rows(statIdLong),timeslots,.)
        ackToAddsg=J(rows(statIdLong),timeslots,.)
        ackToAddre1=J(rows(statIdLong),timeslots,.)
        ackToAddre2=J(rows(statIdLong),timeslots,.)

        for (i=1;i<=rows(looper);i++) {
            statId=looper[i]
            lnewsHat=asarray(Bcs,(statId,1))
            otherlHat=asarray(Bcs,(statId,2))
            nnewsHat=asarray(Bcs,(statId,3))

            siHat=asarray(sharesBcs,(statId,c,1))
            soHat=asarray(sharesBcs,(statId,c,2))
            sgHat=asarray(sharesBcs,(statId,c,3))
            XBVs=asarray(Bcs,(statId,4))
            XBVsm=asarray(Bcs,(statId,5))
            idPos=mm_which(statId:==statIdLong)
            lnewsAct=lnewsLong[idPos,]
            otherlAct=otherlLong[idPos,]
            nnewsAct=nnewsLong[idPos,]
            Mpop=select(popLong,statIdLong:==statId)
            for (t=6;t>=2;t--) {
                if (nnewsAct[,t]!=1) {
                    sameButOneProfs=mm_which((rowsum(lnewsAct[,1::t-1]:==lnewsHat[,1::t-1]):==t-1):*
                        (lnewsAct[,t]:!=lnewsHat[,t]))
                    sameButOneProfs=min(sameButOneProfs)

                    ackToAddsi[idPos,t]=siHat[sameButOneProfs,t]	/* save the shares */
                    ackToAddso[idPos,t]=soHat[sameButOneProfs,t]	
                    ackToAddsg[idPos,t]=sgHat[sameButOneProfs,t]

                    ackToAdd[idPos,t]=XBVs[sameButOneProfs,t]	/* This is wrong! */
                    mu=lnewsHat[sameButOneProfs,t]*bo[1]+
                        otherlHat[sameButOneProfs,t]*bo[2]+
                        nnewsHat[sameButOneProfs,t]*bo[3]+
                        (1-lnewsHat[sameButOneProfs,t]-otherlHat[sameButOneProfs,t]-nnewsHat[sameButOneProfs,t])*bo[4]

                    ackToAddg[idPos,t]=lnnd(XBVs[sameButOneProfs,t],XBVsm[sameButOneProfs,t],bo[22])-ln(1/(1-mu)-
                        mu*siHat[sameButOneProfs,t]/sgHat[sameButOneProfs,t]-siHat[sameButOneProfs,t])
                }
                else {
                    ackToAdd[idPos,t]=0
                    ackToAddg[idPos,t]=0
                    ackToAddsi[idPos,t]=0
                    ackToAddso[idPos,t]=0
                    ackToAddsg[idPos,t]=0
                }
            }
            
            if (nnewsAct[1]!=1) {
                sameButOneProfs=mm_which(lnewsAct[1]:!=lnewsHat[,1])
                sameButOneProfs=min(sameButOneProfs)
                ackToAdd[idPos,1]=XBVs[sameButOneProfs,t]
                mu=lnewsHat[sameButOneProfs,1]*bo[1]+
                    otherlHat[sameButOneProfs,1]*bo[2]+
                    nnewsHat[sameButOneProfs,1]*bo[3]+
                    (1-lnewsHat[sameButOneProfs,1]-otherlHat[sameButOneProfs,1]-nnewsHat[sameButOneProfs,1])*bo[4]			

                ackToAddg[idPos,1]=lnnd(XBVs[sameButOneProfs,1],XBVsm[sameButOneProfs,1],bo[26])-ln(1/(1-mu)-
                    mu*siHat[sameButOneProfs,1]/sgHat[sameButOneProfs,1]-siHat[sameButOneProfs,1])
                ackToAddsi[idPos,1]=siHat[sameButOneProfs,1]
                ackToAddso[idPos,1]=soHat[sameButOneProfs,1]
                ackToAddsg[idPos,1]=sgHat[sameButOneProfs,1]
            }
            else {
                ackToAdd[idPos,1]=0
                ackToAddg[idPos,1]=0
                ackToAddsi[idPos,1]=0
                ackToAddso[idPos,1]=0
                ackToAddsg[idPos,1]=0
            }
            ackToAddre1[idPos,]=UvsLong[idPos,counter::counter+timeslots-1]
            ackToAddre2[idPos,]=UvmtLong[idPos,counter::counter+timeslots-1]
        }
        uv=uv,ackToAdd
        uvg=uvg,ackToAddg
        uvsi=uvsi,ackToAddsi
        uvso=uvso,ackToAddso
        uvsg=uvsg,ackToAddsg
        uvre1=uvre1,ackToAddre1
        uvre2=uvre2,ackToAddre2
        counter=counter+timeslots  
    }
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     for (c=1;c<=draws;c++) {
>         ackToAdd=J(rows(statIdLong),timeslots,.)
>         ackToAddg=J(rows(statIdLong),timeslots,.)
>         ackToAddsi=J(rows(statIdLong),timeslots,.)
>         ackToAddso=J(rows(statIdLong),timeslots,.)
>         ackToAddsg=J(rows(statIdLong),timeslots,.)
>         ackToAddre1=J(rows(statIdLong),timeslots,.)
>         ackToAddre2=J(rows(statIdLong),timeslots,.)
> 
>         for (i=1;i<=rows(looper);i++) {
>             statId=looper[i]
>             lnewsHat=asarray(Bcs,(statId,1))
>             otherlHat=asarray(Bcs,(statId,2))
>             nnewsHat=asarray(Bcs,(statId,3))
> 
>             siHat=asarray(sharesBcs,(statId,c,1))
>             soHat=asarray(sharesBcs,(statId,c,2))
>             sgHat=asarray(sharesBcs,(statId,c,3))
>             XBVs=asarray(Bcs,(statId,4))
>             XBVsm=asarray(Bcs,(statId,5))
>             idP

A secondary block of code computes the sampling weights for the market effects and all that jive. This is because there really isn't all that much to it. Note that what we do is split the density value over all the observations - this just makes computation a little easier as it won't involve finding the last observation of a group or anything like that. 

In [79]:
%%stata
mata:
    uvre1g=lnnd(uvre1,0,ln(sdstav)):*1:/rowsum(gameLong)    
    uvre2Weights=J(rows(statIdLong),timeslots,.)
    for (i=1;i<=rows(mLong);i++) {
        gP=panelsubmatrix(gameLong,i,mLong)
        gPtotal=colsum(gP)
        weightsP=(gPtotal:*gP)
    uvre2Weights[mLong[i,1]::mLong[i,2],.]=weightsP
    }
end    


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     uvre1g=lnnd(uvre1,0,ln(sdstav)):*1:/rowsum(gameLong)    

:     uvre2Weights=J(rows(statIdLong),timeslots,.)

:     for (i=1;i<=rows(mLong);i++) {
>         gP=panelsubmatrix(gameLong,i,mLong)
>         gPtotal=colsum(gP)
>         weightsP=(gPtotal:*gP)
>     uvre2Weights[mLong[i,1]::mLong[i,2],.]=weightsP
>     }

: end


One thing to watch for - it seems that `ipystata` doesn't like it when one puts a little bit of mata code after a loop - not sure why!

In [80]:
%%stata
mata: 
    uvre2g=lnnd(uvre2,0,ln(sdmarv)):*1:/rowsum(uvre2Weights)    
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     uvre2g=lnnd(uvre2,0,ln(sdmarv)):*1:/rowsum(uvre2Weights)    

: end


## Pricing errors

Here, we fill in the pricing errors...which involves a rather tricky recursion. We should flesh out the details about this later. First step: recover actual pricing errors given data and estimated parameters. 

In [81]:
%%stata
mata:
    UpmodObs=J(rows(lnewsLong),0,0)
    for (i=1;i<=draws*timeslots;i=i+timeslots) {
        UpmodObs=UpmodObs,lnppsLong:-lnewsLong:*lnviewnLong:*bpo[1]:-otherlLong:*lnviewnLong:*bpo[2]:-
            nnewsLong:*lnviewnLong:*bpo[3]:-lnewsLong:*bpo[4]:-otherlLong:*bpo[5]:-l_ACS_HHLong:*bpo[6]:-bpo[10]:-
            UpsLong[,i::i+timeslots-1]:-UpmtLong[,i::i+timeslots-1]
    }
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     UpmodObs=J(rows(lnewsLong),0,0)

:     for (i=1;i<=draws*timeslots;i=i+timeslots) {
>         UpmodObs=UpmodObs,lnppsLong:-lnewsLong:*lnviewnLong:*bpo[1]:-otherlLong:*lnviewnLong:*bpo[2]:-
>             nnewsLong:*lnviewnLong:*bpo[3]:-lnewsLong:*bpo[4]:-otherlLong:*bpo[5]:-l_ACS_HHLong:*bpo[6]:-bpo[10]:-
>             UpsLong[,i::i+timeslots-1]:-UpmtLong[,i::i+timeslots-1]
>     }

: end


This code should eventually be posted elsewhere, as it doesn't really belong here. This is an alternative means of calculating errors for the viewership model. (Which we are calling observed errors

In [90]:
%%stata
mata:
    UvmodObsLong4=J(rows(lnewsLong),timeslots*draws,0)
    counter=0
    for (d=1;d<=draws;d++) {
        for (t=1;t<=timeslots;t++) {
            yvt=ln(siLong[,t]):-ln(1:-slnewsLong[,t]:-snnewsLong[,t]:-sotherlLong[,t]:-sothercLong[,t])
            sigmaStuff=bo[1]:*lnswgLong[,t]:*lnewsLong[,t]:+bo[2]:*lnswgLong[,t]:*otherlLong[,t]:+
                bo[3]:*lnswgLong[,t]:*nnewsLong[,t]:+bo[4]:*lnswgLong[,t]:*othercLong[,t]
            XVt=lnewsLong[,t],otherlLong[,t],nnewsLong[,t],
                lnewsLong[,t]:*lnewsLongLag[,t],lnewsLong[,t]:*nnewsLongLag[,t],
                nnewsLong[,t]:*lnewsLongLag[,t],nnewsLong[,t]:*nnewsLongLag[,t],
                siLongLag[,t],siXlnln[,t],siXlnnn[,t],siXnnln[,t],siXnnnn[,t],
                lnewsLong[,t]:*ln(1:+totlnews[,t]),nnewsLong[,t]:*ln(1:+totnnews[,t]),l_ACS_HHLong[,t],
                lnewsnLong[,t],otherlnLong[,t],nnewsnLong[,t],othercnLong[,t],J(rows(lnewsLong),1,1)
            UvmodObsLong4[,counter+t]=yvt:-sigmaStuff:-XVt*betaDynoStart':-UvsLong[,counter+t]:-UvmtLong[,counter+t]		/* This is critical with more expansive RE simulation */
        }
        counter=counter+timeslots
    }
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     UvmodObsLong4=J(rows(lnewsLong),timeslots*draws,0)

:     counter=0

:     for (d=1;d<=draws;d++) {
>         for (t=1;t<=timeslots;t++) {
>             yvt=ln(siLong[,t]):-ln(1:-slnewsLong[,t]:-snnewsLong[,t]:-sotherlLong[,t]:-sothercLong[,t])
>             sigmaStuff=bo[1]:*lnswgLong[,t]:*lnewsLong[,t]:+bo[2]:*lnswgLong[,t]:*otherlLong[,t]:+
>                 bo[3]:*lnswgLong[,t]:*nnewsLong[,t]:+bo[4]:*lnswgLong[,t]:*othercLong[,t]
>             XVt=lnewsLong[,t],otherlLong[,t],nnewsLong[,t],
>                 lnewsLong[,t]:*lnewsLongLag[,t],lnewsLong[,t]:*nnewsLongLag[,t],
>                 nnewsLong[,t]:*lnewsLongLag[,t],nnewsLong[,t]:*nnewsLongLag[,t],
>                 siLongLag[,t],siXlnln[,t],siXlnnn[,t],siXnnln[,t],siXnnnn[,t],
>                 lnewsLong[,t]:*ln(1:+totlnews[,t]),nnewsLong[,t]:*ln(1:+totnnews[,t]),l_ACS_HHLong[,t],
>                 lnews

This next massive loop fills in AKW samplers, but also gets pricing errors, simulated prices, and upper bounds on prices. These are all needed to make the sampler work correctly as the implication is that we have a set of truncated random normal variates. 

Here goes:




In [91]:
%%stata
mata:
    priceErrs=asarray_create("real",2)
    simPrices=asarray_create("real",2)
    priceBounds=asarray_create("real",2)

    up=J(rows(statIdLong),0,.)
    upg=J(rows(statIdLong),0,.)
    upb=J(rows(statIdLong),0,.)
    vup=J(rows(statIdLong),0,.)
    upre1=J(rows(statIdLong),0,.)
    upre2=J(rows(statIdLong),0,.)

    Troublers=J(0,6,.)
    allDraws=J(0,6,.)

    counter=1
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     priceErrs=asarray_create("real",2)

:     simPrices=asarray_create("real",2)

:     priceBounds=asarray_create("real",2)

:     upg=J(rows(statIdLong),0,.)

:     upb=J(rows(statIdLong),0,.)

:     vup=J(rows(statIdLong),0,.)

:     upre1=J(rows(statIdLong),0,.)

:     upre2=J(rows(statIdLong),0,.)

:     allDraws=J(0,6,.)

: end


In [92]:
%%stata
mata:
    for (d=1;d<=draws;d++) {
        uptoAdd=J(rows(statIdLong),6,.)
        uptoAddg=J(rows(statIdLong),6,.)
        uptoAddb=J(rows(statIdLong),6,.)
        vuptoAdd=J(rows(statIdLong),6,.)
        upre1toAdd=J(rows(statIdLong),6,.)
        upre2toAdd=J(rows(statIdLong),6,.)
        gameList=J(0,2,.)
        for (i=1;i<=rows(statIdLong);i++) {
            if (gameMarker[i]==1) {
                idp=statIdLong[i]
                gameList=gameList \ (idp,i)
                lnewsHat=asarray(Bcs,(idp,1))
                otherlHat=asarray(Bcs,(idp,2))
                nnewsHat=asarray(Bcs,(idp,3))
                sharesHat=asarray(sharesBcs,(idp,d,1))
                lnewsAct=lnewsLong[i,]
                otherlAct=otherlLong[i,]
                nnewsAct=nnewsLong[i,]
            
                problem=0 
                Try=1
            
                do {
                    errMarker=(lnewsAct:==lnewsHat):*(otherlAct:==otherlHat)	
                    errPdraws=J(1,6,0)

                    p=mm_which(((rowsum(errMarker[,1::5])):==5):*(errMarker[,timeslots]:==0))	/* Same until last period */

                    actPayMean=(lnewsHat[p,]*bpo[1]:+otherlHat[p,]*bpo[2]:+nnewsHat[p,]*bpo[3]):*ln(popLong[i,]:*sharesHat[p,]):+
                        lnewsHat[p,]:*bpo[4]:+otherlHat[p,]*bpo[5]:+bpo[6]:*l_ACS_HHLong[i,]:+bpo[10]:+
                        UpsLong[i,counter::counter+timeslots-1]:+UpmtLong[i,counter::counter+timeslots-1]
                    Bound=J(1,timeslots,.)	

                    Bound[timeslots]=lnppsLong[i,timeslots]-actPayMean[timeslots]
                    errPdraws[,timeslots]=exp(bpo[9])*invnormal(runiform(1,1)*normal(Bound[timeslots]/exp(bpo[9])))

                    if (nnewsLong[i,timeslots]!=1) {
                        uptoAdd[i,timeslots]=actPayMean[timeslots]+errPdraws[timeslots]
                        meanP=actPayMean[timeslots]
                        uptoAddg[i,timeslots]=lnnd(uptoAdd[i,timeslots],meanP,bpo[9])-
                            ln(normal(Bound[timeslots]/exp(bpo[9])))

                        uptoAddb[i,timeslots]=actPayMean[timeslots]+Bound[timeslots]
                        vuptoAdd[i,timeslots]=ln(popLong[i,1]:*sharesHat[p,timeslots])	/* should be equal to viewership used above */
                    }
                    else {
                        uptoAdd[i,timeslots]=0
                        uptoAddg[i,timeslots]=0
                        uptoAddb[i,timeslots]=0
                        vuptoAdd[i,timeslots]=0
                    }

                    UpmodObz=UpmodObs[i,counter::counter+timeslots-1]
                    for (t=timeslots-1;t>=1;t--) {
                        if (nnewsAct[t]!=1) {
                            if (t>1) {
                                p=mm_which((rowsum(errMarker[,1::t-1]):==t-1):*(errMarker[,t]:==0))
                                useErrs=J(rows(p),timeslots,0)
                                useErrs[,1::t-1]=J(rows(p),1,UpmodObs[i,1::t-1])
                                useErrs[,t+1::cols(useErrs)]=
                                    errMarker[p,t+1::cols(errPdraws)]:*UpmodObz[1,t+1::cols(errPdraws)]:+
                                    (1:-errMarker[p,t+1::cols(errPdraws)]):*errPdraws[,t+1::cols(errPdraws)]
                            }
                            else {
                                p=mm_which(errMarker[,t]:==0)
                                useErrs=J(rows(p),timeslots,0)	
                                useErrs[,t+1::cols(useErrs)]=
                                    errMarker[p,t+1::cols(errPdraws)]:*UpmodObz[1,t+1::cols(errPdraws)]:+
                                    (1:-errMarker[p,t+1::cols(errPdraws)]):*errPdraws[,t+1::cols(errPdraws)]						
                            }

                            actPayMean=(lnewsHat[p,]*bpo[1]:+otherlHat[p,]*bpo[2]:+nnewsHat[p,]*bpo[3]):*ln(popLong[i,]:*sharesHat[p,]):+
                                lnewsHat[p,]:*bpo[4]:+otherlHat[p,]*bpo[5]:+bpo[6]:*l_ACS_HHLong[i,]:+bpo[10]:+
                                UpsLong[i,counter::counter+timeslots-1]:+UpmtLong[i,counter::counter+timeslots-1]

                            max1=sum(exp(lnppsLong[i,t::timeslots]))
                            max2=max(rowsum(exp(actPayMean[,t+1::timeslots]:+useErrs[,t+1::timeslots])))

                            Bound[t]=ln(max1-max2)-actPayMean[rows(actPayMean),t] 		
                            allDraws=allDraws \ (d,i,marketIdLong[i],idp,t,max1-max2)
                            errPdraws[,t]=exp(bpo[9])*invnormal(runiform(1,1)*normal(Bound[t]/exp(bpo[9])))
                            if (errPdraws[,t]==.) {
                                printf("+");displayflush();(d,idp,t)
                                problem=1
                                Troublers=Troublers \ (d,i,marketIdLong[i],idp,t,max1-max2)
                                errPdraws[,t]=-20
                                Bound[t]=-20
                                Try++
                                Try
                            }

                            uptoAdd[i,t]=actPayMean[rows(actPayMean),t]+errPdraws[,t]
                            meanP=actPayMean[rows(actPayMean),t]
                            uptoAddg[i,t]=lnnd(uptoAdd[i,t],meanP,bpo[9])-
                                min((ln(normal((uptoAdd[i,t]-meanP)/exp(bpo[9]))),-ln(normal(-35))))
                            uptoAddb[i,t]=meanP+Bound[t]
                            vuptoAdd[i,t]=min(popLong[i,1]:*sharesHat[p,t])					
                        }
                        else {
                            errPdraws[,t]=UpmodObz[1,t]
                            uptoAdd[i,t]=0
                            uptoAddg[i,t]=0
                            uptoAddb[i,t]=0
                            vuptoAdd[i,t]=0
                        }
                    }
                } while (problem==1 & Try<2)    /* See if it makes a difference here at all */


            asarray(priceErrs,(statIdLong[i],d),errPdraws)
            asarray(priceBounds,(statIdLong[i],d),Bound)

            simPps=(lnewsHat*bpo[1]:+otherlHat*bpo[2]:+nnewsHat*bpo[3]):*ln(sharesHat:*popLong[i,])+
                lnewsHat*bpo[4]:+otherlHat*bpo[5]:+bpo[6]:*l_ACS_HHLong[i,]:+bpo[10]:+
                UpsLong[i,counter::counter+timeslots-1]:+UpmtLong[i,counter::counter+timeslots-1]:+
                errMarker:*UpmodObz[1,1::timeslots]:+(1:-errMarker):*errPdraws
            
            asarray(simPrices,(statIdLong[i],d),simPps)
            
            upre1toAdd[i,]=UpsLong[i,counter::counter+timeslots-1]
            upre2toAdd[i,]=UpmtLong[i,counter::counter+timeslots-1]	
            }
        }
        up=up,uptoAdd
        upg=upg,uptoAddg
        upb=upb,uptoAddb
        vup=vup,vuptoAdd
        upre1=upre1,upre1toAdd
        upre2=upre2,upre2toAdd
        counter=counter+timeslots
    }
end



------------------------------------------------- mata (type end to exit) ------------------------------------------
:     for (d=1;d<=draws;d++) {
>         uptoAdd=J(rows(statIdLong),6,.)
>         uptoAddg=J(rows(statIdLong),6,.)
>         uptoAddb=J(rows(statIdLong),6,.)
>         vuptoAdd=J(rows(statIdLong),6,.)
>         upre1toAdd=J(rows(statIdLong),6,.)
>         upre2toAdd=J(rows(statIdLong),6,.)
>         gameList=J(0,2,.)
>         for (i=1;i<=rows(statIdLong);i++) {
>             if (gameMarker[i]==1) {
>                 idp=statIdLong[i]
>                 gameList=gameList \ (idp,i)
>                 lnewsHat=asarray(Bcs,(idp,1))
>                 otherlHat=asarray(Bcs,(idp,2))
>                 nnewsHat=asarray(Bcs,(idp,3))
>                 sharesHat=asarray(sharesBcs,(idp,d,1))
>                 lnewsAct=lnewsLong[i,]
>                 otherlAct=otherlLong[i,]
>                 nnewsAct=nnewsLong[i,]
>             
>                 problem=0 
>                 Try=1
>

Last thing to do for getting pricing errors together is to make weights for the common random effects. As usual, I just spread these across observations:

In [93]:
%%stata
mata: 
    upre1g=lnnd(upre1,0,ln(sdstap)):*1:/rowsum(gameLong)     /* spread across observations */
    upre2g=lnnd(upre2,0,ln(sdmarp)):*1:/rowsum(uvre2Weights)
end



------------------------------------------------- mata (type end to exit) ------------------------------------------
:     upre1g=lnnd(upre1,0,ln(sdstap)):*1:/rowsum(gameLong)     /* spread across observations */

:     upre2g=lnnd(upre2,0,ln(sdmarp)):*1:/rowsum(uvre2Weights)

: end


## Game Theoretic stuff...

This is far and away the most challenging aspect of this. While we have long loops elsewhere in the program, in this program we have to actually find equilibria for our games when they apply. I won't be too long winded about this now, as it is best to get things started I think...

Note that what we are storing in the following are profits, shares, and the number of profiles. We really haven't used much of this yet, beyond using the Nash profiles to compute AKW sampling weights for each market draw. Note that the scalar given by `actsToTry` holds the number of action profiles for each firm that we consider - the rest are ruled _a priori_ to be too implausible. 

In [94]:
%%stata
mata:
    NashProfiles=asarray_create("real",4)
    NashProfits  =asarray_create("real",3)
    NashShares =asarray_create("real",3)

    PriceMaxSps=asarray_create("real",2) 
    
    actsToTry=10
    
    counter=1
end


------------------------------------------------- mata (type end to exit) ------------------------------------------
:     NashProfiles=asarray_create("real",4)

:     NashProfits  =asarray_create("real",3)

:     NashShares =asarray_create("real",3)

:     
:     actsToTry=10

:     
:     counter=1

: end


Here begins the loop, with a schematic to be written 

In [104]:
%%stata -os

disp "hi"
mata:
    for (d=1;d<=draws;d++) {
    
        for (i=1;i<=rows(mLong);i++) {
            gameMarkerp=panelsubmatrix(gameMarker,i,mLong)
            playersp=colsum(gameMarkerp)

            if (playersp>0) {
                statIdLongp=panelsubmatrix(statIdLong,i,mLong)   
                Gamers=select(statIdLongp,gameMarkerp)           
                posofGamers=mm_which(gameMarkerp)                

                UvmtLongp=panelsubmatrix(UvmtLong,i,mLong)
                UvmodLongp=panelsubmatrix(UvmodLong,i,mLong)
                UvsLongp=panelsubmatrix(UvsLong,i,mLong)
                XBVngLongp=panelsubmatrix(XBVngLong,i,mLong)
                UvmodObsLongp=panelsubmatrix(UvmodObsLong4,i,mLong)

                for (g=1;g<=playersp;g++) {
                    pricesToUse=asarray(simPrices,(Gamers[g],d))
                    sharesToUse=asarray(sharesBcs,(Gamers[g],d,1))
                    maxindex(rowsum(exp(pricesToUse)),actsToTry,iP=.,w=.)
                    asarray(PriceMaxSps,(Gamers[g],d),iP)   
                }

                UpmodObsp=panelsubmatrix(UpmodObs,i,mLong)
                UpsLongp=panelsubmatrix(UpsLong,i,mLong)
                UpmtLongp=panelsubmatrix(UpmtLong,i,mLong)

                lnewsLongp=panelsubmatrix(lnewsLong,i,mLong)
                otherlLongp=panelsubmatrix(otherlLong,i,mLong)
                nnewsLongp=panelsubmatrix(nnewsLong,i,mLong)
                othercLongp=panelsubmatrix(othercLong,i,mLong)
                popLongp=panelsubmatrix(popLong,i,mLong)
                l_ACS_HHLongp=panelsubmatrix(l_ACS_HHLong,i,mLong)

                otherlOld=otherlLongp
                lnewsOld=lnewsLongp
                nnewsOld=nnewsLongp
                othercOld=othercLongp

                neq=1
                asarray(NashProfiles,(i,d,1,neq),lnewsOld)
                asarray(NashProfiles,(i,d,2,neq),otherlOld)
                asarray(NashProfiles,(i,d,3,neq),nnewsOld)

                for (s=1;s<=1000;s++) {
                    targetPlayerN=round(1+(playersp-1)*runiform(1,1))
                    targetPlayer=Gamers[targetPlayerN]
                    
                    lnewsPlayer=asarray(Bcs,(targetPlayer,1))
                    otherlPlayer=asarray(Bcs,(targetPlayer,2))
                    nnewsPlayer=asarray(Bcs,(targetPlayer,3))
                    
                    newStrat=round(1+(rows(lnewsPlayer)-1)*runiform(1,1))

                    targetPos=posofGamers[targetPlayerN]
                    lnewsTry=lnewsOld
                    nnewsTry=nnewsOld
                    otherlTry=otherlOld
                    othercTry=othercOld

                    lnewsTry[targetPos,]=lnewsPlayer[newStrat,]
                    nnewsTry[targetPos,]=nnewsPlayer[newStrat,]
                    otherlTry[targetPos,]=otherlPlayer[newStrat,]
 
                    fail=0
                    g=1     
                    
                    simPpsInit=J(rows(lnewsTry),cols(lnewsTry),0)
                    lnewsInit=lnewsTry
                    nnewsInit=nnewsTry
                    otherlInit=otherlTry
                    othercInit=othercTry
                    do {  
                        rowsToTry=asarray(PriceMaxSps,(Gamers[g],d)) 
                        playerPos=mm_which(statIdLongp:==Gamers[g]) 
                        a=1                 
                        firstTime=1         
                        do {
                            if (firstTime!=1) {
                                lnewsDev=asarray(Bcs,(Gamers[g],1))[rowsToTry[a],]
                                otherlDev=asarray(Bcs,(Gamers[g],2))[rowsToTry[a],]
                                lnewsTry[playerPos,]=lnewsDev
                                otherlTry[playerPos,]=otherlDev
                            }

                            lnewsLongLagp=J(rows(lnewsLongp),1,0)
                            nnewsLongLagp=J(rows(lnewsLongp),1,0)
                            otherlLongLagp=J(rows(lnewsLongp),1,0)
                            siLagp=J(rows(lnewsLongp),1,0)
                            totlnewsp=J(rows(lnewsLongp),1,0)
                            totnnewsp=J(rows(nnewsLongp),1,0)

                            lnewsnTry=lnewsTry:*ln(1:+colsum(lnewsTry))
                            otherlnTry=otherlTry:*ln(1:+colsum(otherlTry))
                            nnewsnTry=nnewsTry:*ln(1:+colsum(nnewsTry))
                            othercnTry=othercTry:*ln(1:+colsum(othercTry))

                            sharesTry=J(rows(lnewsLongp),0,0)
                            for (t=1;t<=timeslots;t++) {
                                if (t!=1) siLagp=ln(siLagp)
                                XV=lnewsTry[,t],otherlTry[,t],nnewsTry[,t],
                                    lnewsLongLagp:*lnewsTry[,t],
                                    nnewsLongLagp:*lnewsTry[,t],
                                    lnewsLongLagp:*nnewsTry[,t], 
                                    nnewsLongLagp:*nnewsTry[,t],siLagp,
                                    siLagp:*lnewsLongLagp:*lnewsTry[,t],
                                    siLagp:*nnewsLongLagp:*lnewsTry[,t],
                                    siLagp:*lnewsLongLagp:*nnewsTry[,t],
                                    siLagp:*nnewsLongLagp:*nnewsTry[,t],
                                    lnewsTry[,t]:*ln(1:+totlnewsp),nnewsTry[,t]:*ln(1:+totnnewsp),l_ACS_HHLongp[,t],
                                    lnewsnTry[,t],otherlnTry[,t],nnewsnTry[,t],othercnTry[,t],J(rows(lnewsLongp),1,1)

                                    errCheck=lnewsTry[,t]:!=lnewsLongp[,t]
                                    XBVact=XV*betaDynoStart':+UvmtLongp[,counter+t-1]:+
                                    UvsLongp[,counter+t-1]:+UvmodObsLongp[,counter+t-1]
                                    XBVsim=XV*betaDynoStart':+UvmtLongp[,counter+t-1]:+
                                        UvsLongp[,counter+t-t]:+UvmodLongp[,counter+t-1]

                                    XBV=(1:-errCheck):*XBVact:+errCheck:*XBVsim

                                    sharesTry=sharesTry,eshares_up(XBV,lnewsTry[,t],otherlTry[,t],nnewsTry[,t],
                                                                   othercLongp[,t],bo[1],bo[2],bo[3],bo[4])

                                    siLagp=sharesTry[,t]
                                    totlnewsp=totlnewsp:+J(rows(lnewsLongp),1,colsum(lnewsTry[,t]:*sharesTry[,t]))
                                    totnnewsp=totnnewsp:+J(rows(nnewsLongp),1,colsum(nnewsTry[,t]:*sharesTry[,t]))
                                    lnewsLongLagp=lnewsTry[,t]
                                    nnewsLongLagp=nnewsTry[,t]
                                    otherlLongLagp=otherlTry[,t]
                            }

                            errCheck=lnewsTry:!=lnewsLongp

                            UpmodErrp=J(rows(UpmodObsp),timeslots,.)

                            for (k=1;k<=playersp;k++) {
                                UpmodErrp[posofGamers[k],]=asarray(priceErrs,(Gamers[k],d))
                            }

                            Errs=errCheck:*UpmodErrp:+(1:-errCheck):*UpmodObsp[,counter::counter+timeslots-1]
                            simPps=(lnewsTry*bpo[1]:+otherlTry*bpo[2]:+nnewsTry*bpo[3]):*ln(sharesTry:*popLongp)+
                                lnewsTry*bpo[4]:+otherlTry*bpo[5]:+l_ACS_HHLongp:*bpo[6]:+bpo[10]:+
                                UpsLongp[,counter::counter+timeslots-1]:+UpmtLongp[,counter::counter+timeslots-1]:+
                                Errs

                            if (rowsum(exp(simPps[playerPos,]))>rowsum(exp(simPpsInit[playerPos,])) & firstTime!=1) fail=1
                            if (firstTime==1) {
                                firstTime=0
                                simPpsInit=simPps
                                sharesInit=sharesTry
                            }
                            else a++
                            
                        } while (a<=actsToTry & fail==0)
                        g++

                    } while (g<=rows(Gamers) & fail==0)
                    if (fail==0) {
                        printf("New Equilibrium Found!\n") 
                        neq++
                        asarray(NashProfiles,(i,d,1,neq),lnewsInit)
                        asarray(NashProfiles,(i,d,2,neq),otherlInit)
                        asarray(NashProfiles,(i,d,3,neq),nnewsInit)
                        asarray(NashShares,(i,d,neq),sharesInit)
                        asarray(NashProfits,(i,d,neq),simPpsInit) 
                        
                    }
                }
            }
        }
    counter=counter+timeslots
        
    fh = fopen("Notifier\record", "rw")
    fwrite(fh, "Something")
    fclose(fh)
    }

end